# 作业一：实现HMM中文分词和BPE英文分词
姓名：邵言

学号：523031910224

## 任务一：HMM模型用于中文分词

任务一评分标准：
1. 共有8处TODO需要填写，共10分。
2. **可编辑代码区域仅限定在TODO的范围内，不允许自行修改其他部分代码。**
3. 用于说明实验的文字和总结不额外计分，但不写会导致扣分。

> 你可以像这样在Markdown单元格中使用引用符号`>`，  
以及在代码单元格中使用注释来说明你的实验。  

---

In [163]:
import pickle
import numpy as np

导入HMM参数，初始化所需的起始概率矩阵、转移概率矩阵、发射概率矩阵，并将它们转换为<b>对数形式</b>。

In [164]:
with open("hmm_parameters.pkl", "rb") as f:
    hmm_parameters = pickle.load(f)

# 非断字（B）为第0行，断字（I）为第1行
# 发射概率矩阵中，词典大小为65536，以汉字的Unicode码点（一个整数值）作为行索引
start_prob_log = np.log(hmm_parameters["start_prob"])  # shape(2,)
trans_matrix_log = np.log(hmm_parameters["trans_mat"])  # shape(2, 2)
emission_matrix_log = np.log(hmm_parameters["emission_mat"])  # shape(2, 65536)

/var/folders/z0/c0f0lkj135z7bz51zvc4b5q00000gn/T/ipykernel_2005/4197557067.py:8: RuntimeWarning: divide by zero encountered in log
  emission_matrix_log = np.log(hmm_parameters["emission_mat"])  # shape(2, 65536)


定义待处理的句子：

In [165]:
# TODO: 将your_name中的xxx替换为你的姓名
your_name = "邵言"

input_sentence = f"{your_name}是一名优秀的学生"

### 1.1 实现Viterbi算法
实现Viterbi算法，并以此进行中文分词。

In [166]:
def viterbi(sent_orig: str, start_prob_log: np.ndarray, trans_mat_log: np.ndarray, emission_mat_log: np.ndarray) -> str:
    """
    Viterbi算法进行中文分词。

    Args:
        sent_orig: str - 输入的句子
        start_prob_log: numpy.ndarray - 起始对数概率矩阵
        trans_mat_log: numpy.ndarray - 转移对数概率矩阵
        emission_mat_log: numpy.ndarray - 发射对数概率矩阵

    Return:
        str - 中文分词的结果
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种标注（B/I）的最大对数概率值
    dp = np.empty((2, len(sent_ord)), dtype=float)

    # `path`用来储存最大概率对应的上步B/I选择
    #  例如 path[1][7] == 1 意味着第8个（从1开始计数）字符标注I对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[0][5] == 1 意味着第6个字符标注B对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[1][1] == 0 意味着第2个字符标注I对应的最大概率，其前一步的隐状态为0（B）
    path = np.zeros((2, len(sent_ord)), dtype=int)

    # [1] TODO: 第一个位置的最大概率值计算【1分】 =================>>>
    dp[0,0] = start_prob_log[0] + emission_matrix_log[0,sent_ord[0]]   # 第一个为0
    dp[1,0] = start_prob_log[1] + emission_matrix_log[1,sent_ord[0]]   # 第一个为1

    # [1] <<<======================= END ==========================

    # [2] TODO: 其余位置的最大概率值计算（填充dp和path矩阵）【2分】 =====>>>
    for i in range(len(sent_ord)-1):
        # 计算转移的条件概率
        prob_trans = np.array([[dp[0,i] + trans_matrix_log[0,0], dp[0,i] + trans_matrix_log[0,1]],
                      [dp[1,i] + trans_matrix_log[1,0], dp[1,i] + trans_matrix_log[1,1]]])

        # argmax，更新path
        if ( prob_trans[0,0] > prob_trans[1,0] ):
            path[0,i+1] = 0
            base_0 = prob_trans[0,0]
        else:
            path[0,i+1] = 1
            base_0 = prob_trans[1,0]
        if ( prob_trans[0,1] > prob_trans[1,1] ):
            path[1,i+1] = 0
            base_1 = prob_trans[0,1]
        else:
            path[1,i+1] = 1
            base_1 = prob_trans[1,1]

        # 更新dp
        dp[0,i+1] = base_0 + emission_matrix_log[0,sent_ord[i+1]]
        dp[1,i+1] = base_1 + emission_matrix_log[1,sent_ord[i+1]]

    # [2] <<<======================= END ==========================

    #  `labels`用来储存每个位置最有可能的隐状态
    labels = [0 for _ in range(len(sent_ord))]

    # [3] TODO: 计算labels每个位置上的值（填充labels矩阵）【1分】 ====>>>
    current = len(sent_ord) - 1
    labels[current] = 0 if dp[0,current] > dp[1,current] else 1
    while current > 0:
        labels[current-1] = path[labels[current],current]
        current -= 1

    # [3] <<<======================= END ==========================

    #  根据labels生成切分好的字符串
    sent_split = []
    for idx, label in enumerate(labels):
        if label == 1:
            sent_split += [sent_ord[idx], ord("/")]
        else:
            sent_split += [sent_ord[idx]]
    sent_split_str = "".join([chr(x) for x in sent_split])

    return sent_split_str

你可以用下述句子测试Viterbi算法的实现：若实现正确，则输出结果应该是`我/爱上/海交/通大/学的/自然/语言/处理/`。

In [167]:
test_sentence = "我爱上海交通大学的自然语言处理"
print("Viterbi算法分词结果：", viterbi(test_sentence, start_prob_log, trans_matrix_log, emission_matrix_log))

Viterbi算法分词结果： 我/爱上/海交/通大/学的/自然/语言/处理/


检查无误后运行下方单元格，对`input_sentence`做分词。

In [168]:
print("Viterbi算法分词结果：", viterbi(input_sentence, start_prob_log, trans_matrix_log, emission_matrix_log))

Viterbi算法分词结果： 邵言/是/一名/优秀/的/学生/


### 1.2 实现前向算法
实现前向算法，计算该句子的对数概率值。

In [169]:
def compute_logprob_by_forward(sent_orig: str, start_prob_log: np.ndarray, trans_mat_log: np.ndarray, emission_mat_log: np.ndarray) -> float:
    """
    前向算法，计算输入中文句子的对数概率值。

    Args:
        sent_orig: str - 输入的句子
        start_prob_log: numpy.ndarray - 起始对数概率矩阵
        trans_mat_log: numpy.ndarray - 转移对数概率矩阵
        emission_mat_log: numpy.ndarray - 发射对数概率矩阵

    Return:
        float - 对数概率值
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，到该位置为止的句子的对数概率
    dp = np.empty((2, len(sent_ord)), dtype=float)

    # [1] TODO: 初始位置概率的计算【1分】 ==========================>>>
    dp[0,0] = start_prob_log[0] + emission_matrix_log[0,sent_ord[0]]    # 第一个为0
    dp[1,0] = start_prob_log[1] + emission_matrix_log[1,sent_ord[0]]    # 第一个为1
    # [1] <<<======================= END ==========================

    ans = None
    # [2] TODO: 先计算其余位置的概率（填充dp矩阵），然后返回对数概率值【2分】 =====>>>
    for i in range(len(sent_ord)-1):
        dp[0,i+1] = np.log(np.exp(dp[0,i] + trans_matrix_log[0,0]) + np.exp(dp[1,i] + trans_matrix_log[1,0])) + emission_matrix_log[0,sent_ord[i+1]]
        dp[1,i+1] = np.log(np.exp(dp[0,i] + trans_matrix_log[0,1]) + np.exp(dp[1,i] + trans_matrix_log[1,1])) + emission_matrix_log[1,sent_ord[i+1]]
    ans = np.log(np.sum(np.exp(dp[:,-1])))
    # [2] <<<======================= END ==========================

    return ans

### 1.3 实现后向算法
实现后向算法，计算该句子的对数概率值。

In [170]:
def compute_logprob_by_backward(sent_orig: str, start_prob_log: np.ndarray, trans_mat_log: np.ndarray, emission_mat_log: np.ndarray) -> float:
    """
    后向算法，计算输入中文句子的对数概率值。

    Args:
        sent_orig: str - 输入的句子
        start_prob_log: numpy.ndarray - 起始对数概率矩阵
        trans_mat_log: numpy.ndarray - 转移对数概率矩阵
        emission_mat_log: numpy.ndarray - 发射对数概率矩阵

    Return:
        float - 对数概率值
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，从结尾到该位置为止的句子的概率
    dp = np.empty((2, len(sent_ord)), dtype=float)

    # [1] TODO: 终末位置概率的初始化【1分】 =================>>>
    dp[0,-1] = 0   # 倒数第一个为0
    dp[1,-1] = 0   # 倒数第一个为1

    # [1] <<<======================= END =====================

    ans = None
    # [2] TODO: 先计算其余位置的概率（填充dp矩阵），然后返回概率值【2分】 =====>>>
    for i in range(len(sent_ord)-1,0,-1):
        dp[0, i-1] = np.log(np.exp(trans_matrix_log[0,0] + emission_matrix_log[0,sent_ord[i]] + dp[0,i]) + np.exp(trans_matrix_log[0,1] + emission_matrix_log[1,sent_ord[i]] + dp[1,i]))
        dp[1, i-1] = np.log(np.exp(trans_matrix_log[1,0] + emission_matrix_log[0,sent_ord[i]] + dp[0,i]) + np.exp(trans_matrix_log[1,1] + emission_matrix_log[1,sent_ord[i]] + dp[1,i]))

    ans = np.log(np.exp(start_prob_log[0] + emission_matrix_log[0,sent_ord[0]] + dp[0,0]) + np.exp(start_prob_log[1] + emission_matrix_log[1,sent_ord[0]] + dp[1,0]))

    # [2] <<<======================= END ==========================

    return ans

如果前向算法与后向算法的实现正确，下面的测试句子所给出的两种算法概率应当几乎相等，约为`-99.9266`。

In [171]:
test_sentence = "我爱上海交通大学的自然语言处理"
print("前向算法概率：", compute_logprob_by_forward(test_sentence, start_prob_log, trans_matrix_log, emission_matrix_log))
print("后向算法概率：", compute_logprob_by_backward(test_sentence, start_prob_log, trans_matrix_log, emission_matrix_log))

前向算法概率： -99.92661770504658
后向算法概率： -99.92661770504657


现在计算`input_sentence`的句子概率值：

In [172]:
print("前向算法概率：", compute_logprob_by_forward(input_sentence, start_prob_log, trans_matrix_log, emission_matrix_log))
print("后向算法概率：", compute_logprob_by_backward(input_sentence, start_prob_log, trans_matrix_log, emission_matrix_log))

前向算法概率： -67.61320986497117
后向算法概率： -67.61320986497115


> 如果你的名字含有生僻字，分词结果以及计算出的句子概率值可能会很“奇怪”。思考一下这是为什么？

### 实验总结
> TODO：请在这里填写实验总结。
- Viterbi算法：找出最佳路径。每一步找出应该走哪一步从上个状态到达当前状态。
- 前向算法与后向算法。把上一步的所有情况当作一个整体，计算下一步各种可能性的概率分布。
    - 注意前向算法和后向算法逻辑上的不同但是结果上的相似，但是实现上略有不同：
        - 前向算法从前开始初始化，往后计算概率，在最后读取结果。每一个状态的概率依赖于上一个状态的两种情况。
        - 后向算法从最后开始初始化（因此初始化为log(1)=0，因为已经是最终的结果，是确定值），往前计算概率，在开头读取结果（还要乘以start_prob的权重）。每一个状态的概率取决于下一个状态的两种情况。

要注意概率和概率对数的正确转换

## 任务二：BPE算法用于英文分词

任务二评分标准：

1. 共有6处TODO需要填写，共10分。
2. **可编辑代码区域仅限定在TODO的范围内，不允许自行修改其他部分代码。**
3. 用于说明实验的文字和总结不额外计分，但不写会导致扣分。

> 你可以像这样在Markdown单元格中使用引用符号`>`，  
以及在代码单元格中使用注释来说明你的实验。  

---

In [173]:
import re
from typing import List, Tuple, Dict
from tqdm import tqdm

导入英语语料库数据。利用这些自然语料，我们将构建一个BPE分词器，对输入文本进行tokenize。

In [174]:
with open("news_2007_en.txt", encoding="utf-8") as f:
    training_corpus = list(map(lambda l: l.strip(), f.readlines()))     # List[str]
for line in training_corpus[:5]:
    print(line)

The journey from the Clyde to Inverness was promoted as the Royal Route after Queen Victoria and Prince Albert made the trip in 1847 and the monarch also sailed on the Gondolier in 1873.
The warning from the Fed, which cut interest rates earlier this week, triggered concern that it might hold off further rate cuts or even consider raising them if inflation accelerates.
When the witness' boyfriend tried to seek help from security, he was jumped by the posse of Smith's boyfriend and left with a fractured face.
There are concrete limits to growth and no one wants to admit that.
KABUL, Afghanistan (CNN) -- Ten people -- including three members of Afghanistan's parliament -- were killed in a suicide bomb blast Tuesday as they visited a sugar plant in Baghlan province, another member of parliament told CNN.


### 2.1 构建单词频次字典
首先将语料中的句子以空格切分成单词，然后将单词拆分成字母加`</w>`的形式，例如`apple`将变为`a p p l e </w>`。请编写相应函数构建BPE算法需要用到的初始状态词典。

In [175]:
ww_splitor_pattern = re.compile(r"[^a-zA-Z']+|(?=')")
_digit_pattern = re.compile(r"\d+")

def white_space_tokenize(corpus: List[str]) -> List[List[str]]:
    """
    先正则化（字母转小写、数字转为N、除去标点符号），然后以空格分词语料中的句子，例如：  
    输入 `corpus = ["I am happy.", "I have 10 apples!"]`，  
    得到 `[["i", "am", "happy"], ["i", "have", "N", "apples"]]`

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        List[List[str]] - 二维List，内部的List由每个句子的单词str构成

    【辅助函数，该函数无需改动】
    """

    tokeneds = [list(filter(lambda token: len(token) > 0, _splitor_pattern.split(_digit_pattern.sub("N", sentence.lower())))) for sentence in corpus]
    return tokeneds

def build_bpe_vocab(corpus: List[str]) -> Dict[str, int]:
    """
    获取语料库中的所有单词，将单词每个字母以空格隔开、结尾加上</w>（单词终止符）后，构建带频数的字典。例如：  
    输入 `corpus = ["I am happy.", "I have 10 apples!"]`，  
    得到
    ```python
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        Dict[str, int] - "单词分词状态->频数"的词典
    """

    tokenized_corpus = white_space_tokenize(corpus)

    bpe_vocab = dict()

    # TODO: 完成函数体【2分】 =============================>>>
    for sentence in tokenized_corpus:
        for word in sentence:
            w = ' '.join(word) + ' </w>'
            bpe_vocab[w] = bpe_vocab.get(w, 0) + 1

    # <<<======================= END ========================

    return bpe_vocab

检查你的实现：

In [176]:
test_bpe_vocab = build_bpe_vocab(["I am happy.", "I have 10 apples!"])
test_bpe_vocab

{'i </w>': 2,
 'a m </w>': 1,
 'h a p p y </w>': 1,
 'h a v e </w>': 1,
 'N </w>': 1,
 'a p p l e s </w>': 1}

### 2.2 构建bigram频次字典
单词频次字典中，每个键都是由空格分隔开的unigram组成的字符串。请统计这些unigram所组成的所有bigram的频次。

In [177]:
def get_bigram_freq(bpe_vocab: Dict[str, int]) -> Dict[Tuple[str, str], int]:
    """
    统计"单词分词状态->频数"的词典中，各bigram的频次（假设该词典中，各个unigram以空格间隔），例如：  
    输入 
    ```python
    bpe_vocab = {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    得到
    ```python
    {
        ('i', '</w>'): 2,
        ('a', 'm'): 1,
        ('m', '</w>'): 1,
        ('h', 'a'): 2,
        ('a', 'p'): 2,
        ('p', 'p'): 2,
        ('p', 'y'): 1,
        ('y', '</w>'): 1,
        ('a', 'v'): 1,
        ('v', 'e'): 1,
        ('e', '</w>'): 1,
        ('N', '</w>'): 1,
        ('p', 'l'): 1,
        ('l', 'e'): 1,
        ('e', 's'): 1,
        ('s', '</w>'): 1
    }
    ```

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        Dict[Tuple[str, str], int] - "bigram->频数"的词典
    """

    bigram_freq = dict()

    # TODO: 完成函数体【1分】 =============================>>>
    for item, count in bpe_vocab.items():
        characters = item.split(" ")
        for i in range(len(characters)-1):
            bigram_freq[(characters[i],characters[i+1])] = bigram_freq.get((characters[i],characters[i+1]),0) + count
    # <<<======================= END ========================

    return bigram_freq

检查你的实现：

In [178]:
test_bigram_freq = get_bigram_freq(test_bpe_vocab)
test_bigram_freq

{('i', '</w>'): 2,
 ('a', 'm'): 1,
 ('m', '</w>'): 1,
 ('h', 'a'): 2,
 ('a', 'p'): 2,
 ('p', 'p'): 2,
 ('p', 'y'): 1,
 ('y', '</w>'): 1,
 ('a', 'v'): 1,
 ('v', 'e'): 1,
 ('e', '</w>'): 1,
 ('N', '</w>'): 1,
 ('p', 'l'): 1,
 ('l', 'e'): 1,
 ('e', 's'): 1,
 ('s', '</w>'): 1}

### 2.3 合并bigram
BPE算法的每轮迭代都会合并出现频率最高的bigram为一个unigram。请实现下述函数，合并单词频次字典中的指定bigram为一个unigram，并更新单词频次字典。
> <b>提示：</b>注意单词频次字典中，每个单词中的unigram都是由空格分隔开的。

In [179]:
def refresh_bpe_vocab_by_merging_bigram(bigram: Tuple[str, str], old_bpe_vocab: Dict[str, int]) -> Dict[str, int]:
    """
    在"单词分词状态->频数"的词典中，合并指定的bigram为单个unigram，最后返回新的词典。例如：  
    输入 
    ```python
    bigram = ('h', 'a'), old_bpe_vocab = {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    得到
    ```python
    {
        'i</w>': 2,
        'a m </w>': 1,
        'ha p p y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    
    Args:
        bigram: Tuple[str, str] - 待合并的bigram
        old_bpe_vocab: Dict[str, int] - 初始"单词分词状态->频数"的词典

    Return:
        Dict[str, int] - 合并后的"单词分词状态->频数"的词典
    """

    new_bpe_vocab = dict()

    # TODO: 完成函数体【1分】 ============================>>>
    for item, count in old_bpe_vocab.items():
        new_bpe_vocab[item.replace(bigram[0]+" "+bigram[1],bigram[0]+bigram[1])] = count

    # <<<======================= END ========================

    return new_bpe_vocab

检查你的实现：

In [180]:
test_new_bpe_vocab = refresh_bpe_vocab_by_merging_bigram(('h', 'a'), test_bpe_vocab)
test_new_bpe_vocab

{'i </w>': 2,
 'a m </w>': 1,
 'ha p p y </w>': 1,
 'ha v e </w>': 1,
 'N </w>': 1,
 'a p p l e s </w>': 1}

### 2.4 获取BPE分词器词表
在充分合并bigram后，单词频次字典中剩下的所有unigram就构成了最终的词表tokens。在这里，我们希望在分词时贪婪地先匹配最长的（长度相同时最常见的）token，因此在实现下列函数时，请将BPE词表按照分词长度降序-出现频次的降序排序顺序返回。

In [181]:
def get_bpe_tokens(bpe_vocab: Dict[str, int]) -> List[Tuple[str, int]]:
    """
    根据"单词分词状态->频数"的词典，返回所得到的BPE分词token词表，并将该列表首先按照token长度降序排序返回，
    token长度相同时再按照出现频次降序排序返回。例如：  
    输入 
    ```python
    bpe_vocab = {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }
    ```
    得到
    ```
    [
        ('i</w>', 2),
        ('ha', 2),
        ('pp', 2),
        ('</w>', 5),
        ('a', 2),
        ('e', 2),
        ('m', 1),
        ('y', 1),
        ('v', 1),
        ('N', 1),
        ('l', 1),
        ('s', 1)
    ]
    ```

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        List[Tuple[str, int]] - BPE分词token和对应频数组成的List
    """

    bpe_tokens_dict = {}
    bpe_tokens = []
    # TODO: 完成函数体【2分】 ============================>>>
    for item,count in bpe_vocab.items():
        characters = item.split(" ")
        for i in characters:
            bpe_tokens_dict[i] = bpe_tokens_dict.get(i,0) + count
    bpe_tokens = [(token, count) for token, count in bpe_tokens_dict.items()]
    bpe_tokens.sort(key = lambda x: (-len(x[0])+x[0].count("</w>")*3,-x[1]))    # 这里要把"</w>"看作一个字符

    # <<<======================= END ========================

    return bpe_tokens

检查你的实现：

In [182]:
test_bpe_vocab = {
    'i</w>': 2,
    'a m </w>': 1,
    'ha pp y </w>': 1,
    'ha v e </w>': 1,
    'N </w>': 1,
    'a pp l e s </w>': 1
}
test_vocab = get_bpe_tokens(test_bpe_vocab)
test_vocab

[('i</w>', 2),
 ('ha', 2),
 ('pp', 2),
 ('</w>', 5),
 ('a', 2),
 ('e', 2),
 ('m', 1),
 ('y', 1),
 ('v', 1),
 ('N', 1),
 ('l', 1),
 ('s', 1)]

### 2.5 对单词进行BPE分词
获取了词表以后，就可以对单词做BPE分词了。按照上述的贪婪规则（先匹配最长的token，长度相同时先匹配最常见的token），请完成下列函数，对输入单词分词并打印出分词结果。

In [183]:
def print_bpe_tokenize(text: str, bpe_tokens: List[Tuple[str, int]]):
    """
    根据按长度降序的BPE分词列表，将所给输入文本进行BPE分词，最后打印结果。
    
    首先对输入的文本（句子）做空白分词，获得单词序列，随后对于一个待BPE分词的单词，  
    按照上述的贪婪规则从列表中寻找BPE分词进行子串匹配，  
    若成功匹配，则对该子串左右的剩余部分递归地进行下一轮匹配，直到剩余部分长度为0，  
    或者剩余部分无法匹配（该部分整体由`"<unknown>"`代替）。
    
    例1：  
    输入 `text = "shanghai"`, `bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]`  
    最终打印 `"sh an g hai </w>"`

    例2：  
    输入 `text = "SU7 in supermarket!"`, `bpe_tokens=[
        ("per", 30),
        ("are", 10),
        ("su", 20),
        ("N", 50),
    ]`  
    最终打印 `"su N <unknown> <unknown> su per <unknown>"`

    Args:
        text: str - 待分词的单词
        bpe_tokens: List[Tuple(str, int)] - BPE分词和对应频数组成的列表
    """
    bpe_words = [word + "</w>" for word in white_space_tokenize([text])[0]]
    def bpe_tokenize(sub_word: str) -> str:
        # TODO: 使用递归函数，定义该分词过程【2分】 =============>>>
        if len(sub_word) == 0: return ""    # 递归终止
        for character in bpe_tokens:
            pos = sub_word.find(character[0])
            if pos != -1:
                return bpe_tokenize(sub_word[:pos]) + " " + character[0] + " " + bpe_tokenize(sub_word[pos+len(character[0]):])
        return "<unknown>"  # OOV
        # <<<======================= END ========================

    res = " ".join([bpe_tokenize(word) for word in bpe_words])
    print(res)

检查你的实现：

In [184]:
print_bpe_tokenize("shanghai", [("hai", 1), ("sh", 1), ("an", 1), ("</w>", 1), ("g", 1)])
print_bpe_tokenize("SU7 in supermarket!", [("per", 30), ("are", 10), ("su", 20), ("N", 50)])

 sh  an  g  hai  </w> 
 su  N <unknown> <unknown>  su  per <unknown>


### 2.6 训练BPE分词器
BPE分词器的训练是迭代地“找到频次最高的bigram-合并为unigram”的过程。请利用好上述编写的函数，对分词器做训练，直至词表达到指定规模。

In [185]:
def train_bpe_vocab(corpus: List[str], target_vocab_size: int) -> List[Tuple[str, int]]:
    """
    训练BPE分词直至词表达到指定大小，得到BPE分词token列表。

    Args:
        corpus: List[str] - 训练语料
        target_vocab_size: int - 目标词表大小

    Return:
        List[Tuple[str, int]] - BPE分词token和对应频数组成的List
    """
    bpe_vocab = build_bpe_vocab(corpus)
    bpe_tokens = get_bpe_tokens(bpe_vocab)
    print("初始BPE词典大小：", len(bpe_tokens))
    with tqdm(desc="训练BPE分词器", total=target_vocab_size) as pbar:
        while len(bpe_tokens) < target_vocab_size:
            # TODO: 完成训练循环内的代码逻辑【2分】 ================>>>
            biagram_freq = get_bigram_freq(bpe_vocab)   # 得到bigram频率
            max_item = max(biagram_freq.items(), key=lambda x: x[1])    # bigram频率最高的两个token
            bpe_vocab = refresh_bpe_vocab_by_merging_bigram(max_item[0],bpe_vocab)  # 合并token，更新bpe_vocab
            bpe_tokens = get_bpe_tokens(bpe_vocab)  # 更新bpe_token
            # <<<======================= END ========================

            pbar.n = len(bpe_tokens)
            pbar.refresh()

    return bpe_tokens

### 测试BPE分词器的分词效果
我们进行两次不同程度的训练：第一次训练仅简单将词表扩充至400，第二次训练将词表扩充至4096。观察一下，不同程度的训练对分词结果有什么影响？

In [186]:
bpe_tokens_1 = train_bpe_vocab(training_corpus, 400)
bpe_tokens_2 = train_bpe_vocab(training_corpus, 4096)

初始BPE词典大小： 29


训练BPE分词器: 439it [00:05, 82.59it/s]                         


初始BPE词典大小： 29


训练BPE分词器: 4303it [00:12, 356.95it/s]                          


In [187]:
test_texts = [
    "naturallanguageprocessing",
    "thecloudslookednonearerthanwheniwaslyingonthestreet",
    "Shanghai Jiao Tong University was founded in 1896.",
    "The quick brown fox jumps over the lazy dog.",
    "I love natural language processing!"
]
print(f"#1 分词器的分词结果为：")
for test_text in test_texts:
    print_bpe_tokenize(test_text, bpe_tokens_1)
print(f"#2 分词器的分词结果为：")
for test_text in test_texts:
    print_bpe_tokenize(test_text, bpe_tokens_2)

#1 分词器的分词结果为：
 n  at  u  r  al  l  an  g  u  a  g  e  p  ro  c  e  s  sing</w> 
 th  ec  l  ou  d  s  l  o  o  k  e  d  n  on  e  arer  th  an  wh  en  i  w  a  s  l  y  in  g  on  th  e  st  re  e  t</w> 
 sh  an  g  h  ai  </w>   j  i  a  o</w>   t  on  g</w>   un  i  v  er  sit  y</w>   w  as</w>   f  oun  d  ed</w>   in</w>   N</w> 
 the</w>   q  u  ic  k</w>   b  row  n  </w>   f  o  x  </w>   j  u  m  p  s</w>   o  v  er</w>   the</w>   l  a  z  y</w>   d  o  g</w> 
 i  </w>   l  o  ve</w>   n  at  u  r  al</w>   l  an  g  u  a  g  e</w>   p  ro  c  e  s  sing</w> 
#2 分词器的分词结果为：
 n  at  ur  al  l  an  g  u  age  proc  e  s  sing</w> 
 the  c  lou  d  s  lo  o  ke  d  none  are  r  than  wh  en  i  w  as  l  y  in  gon  the  stre  e  t</w> 
 shan  ghai</w>   j  i  a  o</w>   t  on  g</w>   un  i  ver  sit  y</w>   was</w>   f  oun  ded</w>   in</w>   N</w> 
 the</w>   qu  ic  k</w>   b  row  n</w>   f  o  x  </w>   j  u  m  p  s</w>   o  ver</w>   the</w>   l  a  z  y</w>   d  o  

### 实验总结
> TODO：请在这里填写实验总结。
从头开始implement了WordPiece算法
按照其定义逐步定义函数即可
要注意
1. 更新bigram的时候，要考虑单词本身的频次
2. </w>的存在让计算字符串长度不能简单地使用len函数。要记得</w>只能看作一个字符